In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 499.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/1

In [1]:
import pandas as pd
from itertools import combinations
from pycaret.regression import *


In [2]:
def info_inicial(df):
    # NOTA: El mapeo de neighborhoods se hace ANTES de llamar a esta función
    # Ya no necesitamos hacerlo aquí

    # Estadísticas de property_price
    precio_promedio = df['property_price'].mean()
    precio_min = df['property_price'].min()
    precio_max = df['property_price'].max()

    # Estadísticas de otros campos
    metros_promedio = df['lot_size'].mean()
    banos_promedio = df['bathroom_count'].mean()
    habitaciones_promedio = df['bedroom_count'].mean()

    # Mostrar resultados
    print(f"Nuḿero de datos:\n{len(df)}")
    print(f"\nPrecio promedio: {precio_promedio/1000:.2f} k€")
    print(f"Precio mínimo: {precio_min/1000:.2f} k€")
    print(f"Precio máximo: {precio_max/1000:.2f} k€")
    print(f"\nPromedio de metros cuadrados: {metros_promedio:.2f} m²")
    print(f"Promedio de baños: {banos_promedio:.2f}")
    print(f"Promedio de habitaciones: {habitaciones_promedio:.2f}")

    return df  # Devuelve el df

def eliminar_outliers(df):
    df_original = df.copy()
    numeric_cols = df.select_dtypes(include='number').columns

    # Outliers in property_price with IQR
    Q1 = df['property_price'].quantile(0.25)
    Q3 = df['property_price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filtering and resetting index
    df = df[(df['property_price'] >= lower_bound) & (df['property_price'] <= upper_bound)].reset_index(drop=True)
    filas_eliminadas = df_original.shape[0] - df.shape[0]

    print(f"\n----Outliers Eliminados ----")
    print(f"Filas eliminadas por outliers en 'property_price': {filas_eliminadas}")

    # Contar outliers in other numeric columns and ensure index alignment
    print("\nValores atípicos en otras columnas numéricas:")
    for col in numeric_cols:
        if col != 'property_price':
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            # Create the boolean mask and explicitly reindex it to match df's index
            outliers_mask = ((df[col] < lower_bound) | (df[col] > upper_bound)).reindex(df.index)
            outliers = df[outliers_mask].shape[0]
            print(f" - {col}: {outliers} valores atípicos")

    return df

def mejor_modelo_pycaret(df, features):
    print('\n-----Evaluando modelos con pycaret------')

    # Verificar qué features faltan
    missing_features = [f for f in features if f not in df.columns]
    if missing_features:
        print(f"\n❌ COLUMNAS FALTANTES: {missing_features}")
        return None

    # Crear el DataFrame para PyCaret
    df_pycaret = df[features + ['property_price']].copy()

    # Verificar si hay valores nulos
    print(f"Valores nulos por columna:")
    null_counts = df_pycaret.isnull().sum()
    print(null_counts[null_counts > 0])

    # Eliminar filas con valores nulos si los hay
    if df_pycaret.isnull().any().any():
        print("Eliminando filas con valores nulos...")
        df_pycaret = df_pycaret.dropna()
        print(f"Filas restantes después de eliminar nulos: {len(df_pycaret)}")

    if len(df_pycaret) < 10:
        print("❌ No hay suficientes datos para entrenar el modelo")
        return None

    setup(
        data=df_pycaret,
        target='property_price',
        session_id=42,
        train_size=0.8,
        normalize=True,
        verbose=False
    )

    # Añade verbose=False para evitar salida innecesaria
    best_model_pycaret = compare_models(sort='RMSE', verbose=False)

    # Usamos pull para capturar la tabla sin mostrarla
    resultados = pull()

    mejor_fila = resultados.loc[resultados['RMSE'].idxmin()]
    nombre_modelo = mejor_fila['Model']
    rmse = mejor_fila['RMSE']

    precio_medio = df['property_price'].mean()
    error_porcentual = (rmse / precio_medio) * 100

    print("=== Mejor Modelo ===")
    print(f"Modelo: {nombre_modelo}")
    print(f"RMSE: {rmse:.2f}")
    print(f"Error porcentual sobre precio promedio: {error_porcentual:.2f}%")

    return best_model_pycaret

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_final = pd.read_csv('/content/drive/MyDrive/TFM/csv/datos_imagenes_con_predicciones_y_aire.csv')


In [7]:
energy_certificate_order = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
energy_certificate_mapping = {cert: i for i, cert in enumerate(energy_certificate_order)}
df_final['energy_certificate_encoded'] = df_final['energy_certificate'].str.lower().map(energy_certificate_mapping)

# Eliminar columna original
df_final = df_final.drop('energy_certificate', axis=1)

# Convertir variables categóricas a 'category'
df_final['property_type'] = df_final['property_type'].astype('category')
df_final['tipo_suelo'] = df_final['tipo_suelo'].astype('category')
df_final['estilo'] = df_final['estilo'].astype('category')

neighborhood_dict = {
    1: 'Chamberí',
    2: 'Centro',
    3: 'Arganzuela',
    4: 'Retiro'
}


In [8]:

# DIAGNÓSTICO: Primero vamos a ver qué columnas están disponibles
print("DIAGNÓSTICO - Columnas disponibles en df_final:")
print(df_final.columns.tolist())
print(f"\nNúmero total de columnas: {len(df_final.columns)}")

# Variables base que siempre se incluyen - VERIFICAR ESTAS PRIMERO
base_features_candidatas = ['bathroom_count', 'bedroom_count', 'floor',
                           'latitude', 'longitude', 'lot_size',
                           'property_type', 'neighborhood', 'exterior', 'ascensor', 'energy_certificate_encoded']

# Variables opcionales para optimizar - VERIFICAR ESTAS TAMBIÉN
optional_features_candidatas = ['aire_acondicionado', 'altura_techo', 'reformado_bin']

# Filtrar solo las que existen realmente
base_features = [col for col in base_features_candidatas if col in df_final.columns]
optional_features = [col for col in optional_features_candidatas if col in df_final.columns]

print(f"\n✅ BASE FEATURES que existen: {base_features}")
print(f"❌ BASE FEATURES que NO existen: {[col for col in base_features_candidatas if col not in df_final.columns]}")

print(f"\n✅ OPTIONAL FEATURES que existen: {optional_features}")
print(f"❌ OPTIONAL FEATURES que NO existen: {[col for col in optional_features_candidatas if col not in df_final.columns]}")

if not base_features:
    print("\n🚨 ERROR: No hay features base válidas!")
    exit()

DIAGNÓSTICO - Columnas disponibles en df_final:
['address', 'agency_name', 'bathroom_count', 'bedroom_count', 'floor', 'latitude', 'longitude', 'lot_size', 'property_description', 'property_id', 'property_images', 'property_price', 'property_title', 'property_type', 'neighborhood', 'exterior', 'ascensor', 'altura_techo', 'tipo_suelo', 'estilo', 'dist_metro_m', 'aire_acondicionado', 'reformado_bin', 'energy_certificate_encoded']

Número total de columnas: 24

✅ BASE FEATURES que existen: ['bathroom_count', 'bedroom_count', 'floor', 'latitude', 'longitude', 'lot_size', 'property_type', 'neighborhood', 'exterior', 'ascensor', 'energy_certificate_encoded']
❌ BASE FEATURES que NO existen: []

✅ OPTIONAL FEATURES que existen: ['aire_acondicionado', 'altura_techo', 'reformado_bin']
❌ OPTIONAL FEATURES que NO existen: []


In [13]:
from itertools import combinations
import joblib

def probar_combinaciones_variables(df_barrio, base_features, optional_features):
    """
    Prueba todas las combinaciones posibles de variables opcionales
    y devuelve la combinación que da el mejor RMSE, guardando el mejor modelo.
    """
    mejores_features = base_features.copy()
    mejor_rmse = float('inf')
    mejor_modelo = None
    mejor_nombre_modelo = None
    resultados = []

    print("Probando diferentes combinaciones de variables:")

    # Generar todas las combinaciones posibles (incluyendo conjunto vacío)
    for r in range(len(optional_features) + 1):
        for combo in combinations(optional_features, r):
            features_actuales = base_features + list(combo)
            combo_str = ", ".join(combo) if combo else "Solo variables base"

            try:
                # Capturar salida temporalmente
                import io
                import sys
                old_stdout = sys.stdout
                sys.stdout = captured_output = io.StringIO()

                try:
                    resultado_modelo = mejor_modelo_pycaret(df_barrio, features_actuales)
                finally:
                    sys.stdout = old_stdout

                # Obtener RMSE, modelo y nombre
                rmse_actual, modelo_actual, nombre_modelo_actual = obtener_rmse_y_modelo(resultado_modelo)

                print(f"  - {combo_str}: RMSE = {rmse_actual:.4f} | Modelo: {nombre_modelo_actual}")

                resultados.append({
                    'combinacion': combo,
                    'features': features_actuales,
                    'rmse': rmse_actual,
                    'modelo': modelo_actual,
                    'nombre_modelo': nombre_modelo_actual
                })

                # Actualizar mejor combinación si es necesario
                if rmse_actual < mejor_rmse:
                    mejor_rmse = rmse_actual
                    mejores_features = features_actuales.copy()
                    mejor_modelo = modelo_actual
                    mejor_nombre_modelo = nombre_modelo_actual

            except Exception as e:
                print(f"  - {combo_str}: Error - {str(e)}")

    return mejores_features, mejor_rmse, mejor_modelo, mejor_nombre_modelo, resultados


In [16]:
def obtener_rmse_y_modelo(resultado_modelo):
    """
    Función auxiliar que devuelve el RMSE, el modelo y el nombre del modelo desde PyCaret.
    """
    try:
        from pycaret.regression import pull

        modelo = resultado_modelo
        resultados = pull()

        rmse = resultados.iloc[0]['RMSE']
        nombre_modelo = resultados.iloc[0]['Model']

        return rmse, modelo, nombre_modelo

    except Exception as e:
        print(f"Error obteniendo RMSE: {e}")
        return float('inf'), None, "Desconocido"


In [17]:

# Crear una lista para almacenar los DataFrames procesados de cada barrio
df_procesados = []
resultados_por_barrio = {}

for codigo, nombre in neighborhood_dict.items():
    print(f"\n================= Análisis para {nombre.upper()} =================")

    # 1. Filtrar usando el código numérico
    df_barrio = df_final[df_final['neighborhood'] == codigo].copy()

    print(f"Datos encontrados para {nombre}: {len(df_barrio)}")

    if len(df_barrio) < 10:
        print(f"❌ Insuficientes datos para {nombre} (solo {len(df_barrio)} registros)")
        continue

    # 2. Aplicar mapeo de nombres
    df_barrio['neighborhood'] = df_barrio['neighborhood'].map(neighborhood_dict)

    # Verificar que el mapeo funcionó
    print(f"Valores únicos en 'neighborhood' después del mapeo: {df_barrio['neighborhood'].unique()}")

    # 3. Info inicial
    df_barrio = info_inicial(df_barrio)

    # 4. Eliminar outliers
    df_barrio = eliminar_outliers(df_barrio)

    if len(df_barrio) < 10:
        print(f"❌ Insuficientes datos para {nombre} después de eliminar outliers")
        continue

    # 5. Selección automática de mejores variables
    print('\n----- Optimización de variables -----')
    mejores_features, mejor_rmse, mejor_modelo, mejor_nombre_modelo, todos_resultados = probar_combinaciones_variables(
    df_barrio, base_features, optional_features
)
    nombre_archivo_modelo = f'modelo_{nombre.lower().replace(" ", "_")}.pkl'
    joblib.dump(mejor_modelo, nombre_archivo_modelo)
    print(f"📦 Modelo guardado como: {nombre_archivo_modelo}")


    # Mostrar resultados de la optimización
    variables_seleccionadas = [var for var in mejores_features if var in optional_features]
    variables_excluidas = [var for var in optional_features if var not in mejores_features]

    print(f"\n✓ Mejor combinación para {nombre}:")
    print(f"  - RMSE: {mejor_rmse:.4f}")
    print(f"  - Variables adicionales incluidas: {variables_seleccionadas if variables_seleccionadas else 'Ninguna'}")
    print(f"  - Variables adicionales excluidas: {variables_excluidas if variables_excluidas else 'Ninguna'}")

    # Guardar resultados para análisis posterior
    resultados_por_barrio[nombre] = {
    'mejores_features': mejores_features,
    'mejor_rmse': mejor_rmse,
    'mejor_modelo': mejor_modelo,
    'nombre_modelo': mejor_nombre_modelo,  # ← AÑADIDO
    'todos_resultados': todos_resultados,
    'variables_adicionales_incluidas': variables_seleccionadas,
    'variables_adicionales_excluidas': variables_excluidas
}


    # Agregar el DataFrame procesado a la lista
    df_procesados.append(df_barrio)



================= Análisis para CHAMBERÍ =================
Datos encontrados para Chamberí: 365
Valores únicos en 'neighborhood' después del mapeo: ['Chamberí']
Nuḿero de datos:
365

Precio promedio: 1317.66 k€
Precio mínimo: 250.00 k€
Precio máximo: 3400.00 k€

Promedio de metros cuadrados: 149.39 m²
Promedio de baños: 2.35
Promedio de habitaciones: 2.94

----Outliers Eliminados ----
Filas eliminadas por outliers en 'property_price': 18

Valores atípicos en otras columnas numéricas:
 - bathroom_count: 0 valores atípicos
 - bedroom_count: 1 valores atípicos
 - floor: 3 valores atípicos
 - latitude: 0 valores atípicos
 - longitude: 0 valores atípicos
 - lot_size: 6 valores atípicos
 - property_id: 39 valores atípicos
 - exterior: 70 valores atípicos
 - ascensor: 22 valores atípicos
 - altura_techo: 29 valores atípicos
 - dist_metro_m: 0 valores atípicos
 - aire_acondicionado: 0 valores atípicos
 - reformado_bin: 0 valores atípicos
 - energy_certificate_encoded: 34 valores atípicos

---

In [18]:

# Combinar todos los DataFrames procesados en uno solo
df_final_procesado = pd.concat(df_procesados, ignore_index=True)

# Resumen final de optimización
print("\n" + "="*80)
print("RESUMEN FINAL DE OPTIMIZACIÓN POR BARRIO")
print("="*80)

for nombre, resultados in resultados_por_barrio.items():
    print(f"\n{nombre}:")
    print(f"  - RMSE final: {resultados['mejor_rmse']:.4f}")
    print(f"  - Variables adicionales utilizadas: {resultados['variables_adicionales_incluidas'] if resultados['variables_adicionales_incluidas'] else 'Ninguna'}")

    # Mostrar mejora vs modelo base si disponible
    modelo_base_result = next((r for r in resultados['todos_resultados'] if len(r['combinacion']) == 0), None)
    if modelo_base_result and len(resultados['variables_adicionales_incluidas']) > 0:
        mejora = modelo_base_result['rmse'] - resultados['mejor_rmse']
        mejora_pct = (mejora / modelo_base_result['rmse']) * 100
        print(f"  - Mejora vs modelo base: {mejora:.4f} ({mejora_pct:.2f}%)")

# Análisis global de variables más útiles
print(f"\n{'-'*50}")
print("ANÁLISIS GLOBAL DE VARIABLES:")
print(f"{'-'*50}")

conteo_variables = {}
for var in optional_features:
    conteo_variables[var] = sum(1 for r in resultados_por_barrio.values() if var in r['variables_adicionales_incluidas'])

for var, count in conteo_variables.items():
    porcentaje = (count / len(neighborhood_dict)) * 100
    print(f"- {var}: Útil en {count}/{len(neighborhood_dict)} barrios ({porcentaje:.1f}%)")


RESUMEN FINAL DE OPTIMIZACIÓN POR BARRIO

Chamberí:
  - RMSE final: 211055.1762
  - Variables adicionales utilizadas: ['reformado_bin']
  - Mejora vs modelo base: 10348.2122 (4.67%)

Centro:
  - RMSE final: 292559.2054
  - Variables adicionales utilizadas: ['aire_acondicionado']
  - Mejora vs modelo base: 4792.5125 (1.61%)

Arganzuela:
  - RMSE final: 70692.2847
  - Variables adicionales utilizadas: ['reformado_bin']
  - Mejora vs modelo base: 1421.2788 (1.97%)

Retiro:
  - RMSE final: 170435.7021
  - Variables adicionales utilizadas: ['altura_techo']
  - Mejora vs modelo base: 6602.6406 (3.73%)

--------------------------------------------------
ANÁLISIS GLOBAL DE VARIABLES:
--------------------------------------------------
- aire_acondicionado: Útil en 1/4 barrios (25.0%)
- altura_techo: Útil en 1/4 barrios (25.0%)
- reformado_bin: Útil en 2/4 barrios (50.0%)
